In [13]:
import nltk

stopword_list = nltk.corpus.stopwords.words('english')
stopword_list = stopword_list + ['mr', 'ms', 'come', 'go', 'get', 'tell', 'listen', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'zero', 'join', 'find', 'make', 'say', 'ask', 'tell', 'see', 'try', 'back', 'also']

In [14]:
import re

def keep_text_characters(text):
    filtered_tokens = []
    tokens = tokenize_text(text)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

In [15]:
from module.contractions import expand_contractions
from module.lemmatize import lemmatize_text

def normalize_corpus(corpus, lemmatize=True,
                     only_text_chars=False,
                     tokenize=False):
    normalized_corpus = []
    for text in corpus:
        text = html.unescape(text)
        text = expand_contractions(text)
    if lemmatize:
        text = lemmatize_text(text)
    else:
        text = text.lower()
    text = remove_special_characters(text)
    text = remove_stopwords(text)
    if only_text_chars:
        text = keep_text_characters(text)
    if tokenize:
        text = tokenize_text(text)
        normalized_corpus.append(text)
    else:
        normalized_corpus.append(text)
    return normalized_corpus

In [16]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

def build_feature_matrix(documents, feature_type='frequency', ngram_range=(1,1), min_df=0.0, max_df=1.0):
    feature_type = feature_type.lower().strip()
    
    if feature_type == 'binary':
        vectorizer = CountVectorizer(binary=True, min_df=min_df,
                                     max_df=max_df, 
                                     ngram_range=ngram_range)
    elif feature_type == 'frequency':
        vectorizer = CountVectorizer(binary=False, min_df=min_df,
                                     max_df=max_df, ngram_range=ngram_range)
    elif feature_type == 'tfidf':
        vectorizer = TfidfVectorizer(min_df=min_df, max_df=max_df,
                                     ngram_range=ngram_range)
    else:
        raise Exception('Wrong feature type entered. Possible values: "binary", "frequency", "tfidf".')
        
    feature_matrix = vectorizer.fit_transform(documents).astype(float)
    
    return vectorizer, feature_matrix

# Text similarity

Analyzing text similarity:

- lexical similarity: observing the contents of the text documents with regard to syntax, structure and content and measuring their similarity based on these parameters
- semantic similarity: trying to find out the semantics, meaning and context of the documents and then trying to see how close they are to each other. Dependency grammars and entity recognition are handy tools that can help in this.

Lexical similarity is more straightforward to implement.

Two broad areas of text similarity:
- term similarity: measure similarity between individual tokens or words
- document similarity: measure similarity between entire text documents

## Analyzing term similarity

Word representations
- character vectorization
- bag of characters vectorization

In [17]:
import numpy as np

def vectorize_terms(terms):
    terms = [term.lower() for term in terms]
    terms = [np.array(list(term)) for term in terms]
    terms = [np.array([ord(char) for char in term])
             for term in terms]
    return terms

In [18]:
# Bag of character vectorization is similar to the bag of words model except here we compute the frequency of each character in the word.

def boc_term_vectors(word_list):
    word_list = [word.lower() for word in word_list]
    unique_chars = np.unique(
     np.hstack([list(word)
                for word in word_list]))
    word_list_term_counts = [{char: count for char, count in zip(*np.unique(list(word), return_counts=True))} for word in word_list]
    boc_vectors = [np.array([int(word_term_counts.get(char, 0))
                             for char in unique_chars])
                   for word_term_counts in word_list_term_counts]

    return list(unique_chars), boc_vectors

In [19]:
root = 'Believe'
term1 = 'beleive'
term2 = 'bargain'
term3 = 'Elephant'

terms = [root, term1, term2, term3]

In [20]:
# Character vectorization.
vec_root, vec_term1, vec_term2, vec_term3 = vectorize_terms(terms)
vec_root, vec_term1, vec_term2, vec_term3

(array([ 98, 101, 108, 105, 101, 118, 101]),
 array([ 98, 101, 108, 101, 105, 118, 101]),
 array([ 98,  97, 114, 103,  97, 105, 110]),
 array([101, 108, 101, 112, 104,  97, 110, 116]))

In [21]:
# Bag of characters vectorization.
features, (boc_root, boc_term1, boc_term2, boc_term3) = boc_term_vectors(terms)
print('Features')
features, (boc_root, boc_term1, boc_term2, boc_term3)

Features


(['a', 'b', 'e', 'g', 'h', 'i', 'l', 'n', 'p', 'r', 't', 'v'],
 (array([0, 1, 3, 0, 0, 1, 1, 0, 0, 0, 0, 1]),
  array([0, 1, 3, 0, 0, 1, 1, 0, 0, 0, 0, 1]),
  array([2, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0]),
  array([1, 0, 2, 0, 1, 0, 1, 1, 1, 0, 1, 0])))

# Measuring similarity distance

- hamming distance
- manhattan distance
- euclidean distance
- levenshtein edit distance
- cosine distance and similarity

In [22]:
root_term = root
root_vector = vec_root
root_boc_vector = boc_root

terms = [term1, term2, term3]
vector_terms = [vec_term1, vec_term2, vec_term3]
boc_vector_terms = [boc_term1, boc_term2, boc_term3]

# Hamming distance

Distance measured between two strings under the assumptions that they are of equal length. Hamming distance is defined as the number of positions that have different characters or symbols between two strings of equal length.

In [23]:
def hamming_distance(u, v, norm=False):
    if u.shape != v.shape:
        raise ValueError('The vectors must have equal length')
    return (u != v).sum() if not norm else (u != v).mean()

In [24]:
# Compute hamming distance.
for term, vector_term in zip(terms, vector_terms):
    print(f'Hamming distance between root: {root_term} and term {term} is: {hamming_distance(root_vector, vector_term, norm=False)}')

Hamming distance between root: Believe and term beleive is: 2
Hamming distance between root: Believe and term bargain is: 6


ValueError: The vectors must have equal length

In [25]:
# Compute normalized Hamming distance.
for term, vector_term in zip(terms, vector_terms):
    print(f'Hamming distance between root: {root_term} and term {term} is: {round(hamming_distance(root_vector, vector_term, norm=True), 2)}')

Hamming distance between root: Believe and term beleive is: 0.29
Hamming distance between root: Believe and term bargain is: 0.86


ValueError: The vectors must have equal length

# Manhattan distance

Also known as _city block distance_, _L1 norm_, _taxicab metric_ and is defined as the distance between two points in a grid based on strictly horizontal or vertical paths instead of the diagonal distance conventionally calculated by the Euclidean distance metric.

In [26]:
def manhattan_distance(u, v, norm=False):
    if u.shape != v.shape:
        raise ValueError('The vectors must have equal lengths.')
    return abs(u - v).sum() if not norm else abs(u - v).mean()

In [27]:
# Compute Manhattan distance.
for term, vector_term in zip(terms, vector_terms):
    print(f'Manhattan distance between root: {root_term} and term {term} is: {manhattan_distance(root_vector, vector_term, norm=False)}')

Manhattan distance between root: Believe and term beleive is: 8
Manhattan distance between root: Believe and term bargain is: 38


ValueError: The vectors must have equal lengths.

In [28]:
# Compute normalized Manhattan distance.
for term, vector_term in zip(terms, vector_terms):
    print(f'Manhattan distance between root: {root_term} and term {term} is: {round(manhattan_distance(root_vector, vector_term, norm=True), 2)}')

Manhattan distance between root: Believe and term beleive is: 1.14
Manhattan distance between root: Believe and term bargain is: 5.43


ValueError: The vectors must have equal lengths.

# Euclidean distance

Euclidean distance is also known as the _euclidean norm_, _l2 norm_, or _l2 distance_ and is defined as the shortest straight line distance between two points.

In [29]:
def euclidean_distance(u, v):
    if u.shape != v.shape:
        raise ValueError('The vectors must have equal lengths.')
    distance = np.sqrt(np.sum(np.square(u - v)))
    return distance

In [30]:
# Compute euclidean distance.
for term, vector_term in zip(terms, vector_terms):
    print('Euclidean distance between root {} and term {} is {}'.format(root_term,
                                                                        term,
                                                                        round(euclidean_distance(root_vector, vector_term), 2)))

Euclidean distance between root Believe and term beleive is 5.66
Euclidean distance between root Believe and term bargain is 17.94


ValueError: The vectors must have equal lengths.

# Levenshtein edit distance

Defined as the minimum number of edits needed in the form of additions, deletions or substitutions to change or convert one term to the other. The length of two terms need not be equal here.

In [31]:
import copy
import pandas as pd

def levenshtein_edit_distance(u, v):
    # Convert to lower case.
    u = u.lower()
    v = v.lower()
    
    # Base cases.
    if u == v: return 0
    elif len(u) == 0: return len(v)
    elif len(v) == 0: return len(u)
    
    # Initialize edit distance matrix.
    edit_matrix = []
    
    # Initialize two distance matrices.
    du = [0] * (len(v) + 1)
    dv = [0] * (len(v) + 1)
    
    # du: the previous row of edit distances.
    for i in range(len(du)):
        du[i] = i
        
    # dv: the current row of edit distances.
    for i in range(len(u)):
        dv[0] = i + 1
        
        # Compute cost as per algorithm.
        for j in range(len(v)):
            cost = 0 if u[i] == v[j] else 1
            dv[j + 1] = min(dv[j] + 1, du[j + 1] + 1, du[j] + cost)
        
        # Assign dv to du for next iteration.
        for j in range(len(du)):
            du[j] = dv[j]
        
        # Copy dv to the edit matrix.
        edit_matrix.append(copy.copy(dv))
        
    # Compute the final edit distance and edit matrix.
    distance = dv[len(v)]
    edit_matrix = np.array(edit_matrix)
    edit_matrix = edit_matrix.T
    edit_matrix = edit_matrix[1:,]
    edit_matrix = pd.DataFrame(data=edit_matrix,
                               index=list(v),
                               columns=list(u))
    return distance, edit_matrix

In [32]:
for term in terms:
    edit_d, edit_m = levenshtein_edit_distance(root_term, term)
    print(f'Computing distance between root: {root_term} and term: {term}')
    print(f'Levenshtein edit distance is {edit_d}')
    print('The complete edit distance matrix is depicted below')
    print(edit_m)
    print('-' * 30)

Computing distance between root: Believe and term: beleive
Levenshtein edit distance is 2
The complete edit distance matrix is depicted below
   b  e  l  i  e  v  e
b  0  1  2  3  4  5  6
e  1  0  1  2  3  4  5
l  2  1  0  1  2  3  4
e  3  2  1  1  1  2  3
i  4  3  2  1  2  2  3
v  5  4  3  2  2  2  3
e  6  5  4  3  2  3  2
------------------------------
Computing distance between root: Believe and term: bargain
Levenshtein edit distance is 6
The complete edit distance matrix is depicted below
   b  e  l  i  e  v  e
b  0  1  2  3  4  5  6
a  1  1  2  3  4  5  6
r  2  2  2  3  4  5  6
g  3  3  3  3  4  5  6
a  4  4  4  4  4  5  6
i  5  5  5  4  5  5  6
n  6  6  6  5  5  6  6
------------------------------
Computing distance between root: Believe and term: Elephant
Levenshtein edit distance is 7
The complete edit distance matrix is depicted below
   b  e  l  i  e  v  e
e  1  1  2  3  4  5  6
l  2  2  1  2  3  4  5
e  3  2  2  2  2  3  4
p  4  3  3  3  3  3  4
h  5  4  4  4  4  4  4
a  6 

# Cosine distance and similarity

The cosine distance is a metric that can be actually derived from the cosine similarity and vice versa. Considering we have two terms such that they are represented in their vectorized forms, Cosine similarity gives us the measure of the cosine of the angle between them when they are represented as non-zero positive vectors in an inner product space.

Term vectors having similar orientation will have scores closer to 1 (cos 0) indicating the vectors are very close to each other in the same direction (near to zero degree angle between them). Term vectors having a similarity score close to 0 (cos 90) indicate unrelated terms with a near orthogonal angle between them. Term vectors with a similarity score close to -1 (cos 180) indicate terms that are completely oppositely oriented to each other.

In [33]:
def cosine_distance(u, v):
    distance = 1.0 - (np.dot(u, v) / 
                      (np.sqrt(sum(np.square(u))) * np.sqrt(sum(np.square(v))))
                     )
    return distance

In [34]:
for term, boc_term in zip(terms, boc_vector_terms):
    print('Analyzing similarity between root: {} and term: {}'.format(root_term, term))
    distance = round(cosine_distance(root_boc_vector, boc_term), 2)
    similarity = 1 - distance
    print(f'Cosine distance is {distance}')
    print(f'Cosine similarity is {similarity}')
    print('-' * 40)

Analyzing similarity between root: Believe and term: beleive
Cosine distance is -0.0
Cosine similarity is 1.0
----------------------------------------
Analyzing similarity between root: Believe and term: bargain
Cosine distance is 0.82
Cosine similarity is 0.18000000000000005
----------------------------------------
Analyzing similarity between root: Believe and term: Elephant
Cosine distance is 0.39
Cosine similarity is 0.61
----------------------------------------


# Analyzing document similarity

Metrics:
- cosine similarity
- hellinger-bhattacharya distance
- okapi bm25 ranking


In [44]:
from module.utils import normalize_corpus, build_feature_matrix
import numpy as np

In [45]:
# Load the toy corpus index.
toy_corpus = [
    'The sky is blue',
    'The sky is blue and beautiful',
    'Look at the bright blue sky!',
    'Python is a great programming language',
    'Python and Java are popular Programming languages',
    'Among Programming languages, both Python and Java are the most used in Analytics',
    'The fox is quicker than the lazy dog',
    'The dog is smarter than the fox',
    'The dog, fox and cat are good friends'
]

In [46]:
# Load the docs for which we will be measuring similarities.
query_docs = ['The fox is definitely smarter than the dog',
              'Java is a static typed programming language unlike Python',
              'I love to relax under the beautiful blue sky!']

In [47]:
# Normalize and extract features from the toy corpus.
norm_corpus = normalize_corpus(toy_corpus, lemmatize=True)
tfidf_vectorizer, tfidf_features = build_feature_matrix(norm_corpus, 
                                                        feature_type='tfidf')

normlaize


In [49]:
# Normalize and extract features from the query corpys.
norm_query_docs = normalize_corpus(query_docs, lemmatize=True)
query_docs_tfidf = tfidf_vectorizer.transform(norm_query_docs)

normlaize


<3x22 sparse matrix of type '<class 'numpy.float64'>'
	with 10 stored elements in Compressed Sparse Row format>

# Cosine similarity.

In [50]:
def compute_cosine_similarity(doc_features, corpus_features,
                              top_n=3):
    # Get document vectors.
    doc_features = doc_features.toarray()[0]
    corpus_features = corpus_features.toarray()
    
    # Compute similarities.
    similarity = np.dot(doc_features,
                        corpus_features.T)
    
    # Get doc with highest similarity scores.
    top_docs = similarity.argsort()[::-1][:top_n]
    top_docs_with_score = [(index, round(similarity[index], 3))
                           for index in top_docs]
    return top_docs_with_score

In [51]:
# Get Cosine similarity results for our example documents.
print('Document similarity analysis using cosine similarity')
print('=' * 60)

for index, doc in enumerate(query_docs):
    doc_tfidf = query_docs_tfidf[index]
    top_similar_docs = compute_cosine_similarity(doc_tfidf, 
                                                 tfidf_features,
                                                 top_n=2)
    
    print(f'Document {index+1}: {doc}')
    print(f'Top {len(top_similar_docs)} similar docs:')
    print('-' * 40)
    for doc_index, sim_score in top_similar_docs:
        print(f'Doc num: {doc_index+1} Similarity score: {sim_score}\nDoc: {toy_corpus[doc_index]}')
        print('-' * 40)
    print()

Document similarity analysis using cosine similarity
Document 1: The fox is definitely smarter than the dog
Top 2 similar docs:
----------------------------------------
Doc num: 8 Similarity score: 1.0
Doc: The dog is smarter than the fox
----------------------------------------
Doc num: 7 Similarity score: 0.426
Doc: The fox is quicker than the lazy dog
----------------------------------------

Document 2: Java is a static typed programming language unlike Python
Top 2 similar docs:
----------------------------------------
Doc num: 5 Similarity score: 0.837
Doc: Python and Java are popular Programming languages
----------------------------------------
Doc num: 6 Similarity score: 0.661
Doc: Among Programming languages, both Python and Java are the most used in Analytics
----------------------------------------

Document 3: I love to relax under the beautiful blue sky!
Top 2 similar docs:
----------------------------------------
Doc num: 2 Similarity score: 1.0
Doc: The sky is blue and

# Hellinger-Bhattacharya Distance

Aka Hellinger Distance or the Bhattacharya distance, is used to measure the similarity between two discrete or continuous probability distributions.

In [53]:
def compute_hellinger_bhattacharya_distance(doc_features, corpus_features, top_n=3):
    # Get document vectors.
    doc_features = doc_features.toarray()[0]
    corpus_features = corpus_features.toarray()
    
    # Compute hb distances.
    distance = np.hstack(
     np.sqrt(0.5 * np.sum(np.square(np.sqrt(doc_features) - np.sqrt(corpus_features)),
                          axis=1)))
    
    # Get docs with the lowest distance scores.
    top_docs = distance.argsort()[:top_n]
    top_docs_with_scores = [(index, round(distance[index], 3))
                             for index in top_docs]
    return top_docs_with_scores

In [54]:
# Get Hellinger-Bhattacharya distance based similarities for our example documents.
print('Document similarity analysis using Hellinger-Bhattacharya distance:')
print('=' * 60)
for index, doc in enumerate(query_docs):
    doc_tfidf = query_docs_tfidf[index]
    top_similar_docs = compute_hellinger_bhattacharya_distance(doc_tfidf, 
                                                               tfidf_features,
                                                               top_n=2)
    print(f'Document {index+1}: {doc}')
    print(f'Top {len(top_similar_docs)} similar docs')
    print('-' * 40)
    for doc_index, sim_score in top_similar_docs:
        print(f'Doc num: {doc_index+1}: Distance score: {sim_score}\nDoc: {toy_corpus[doc_index]}')
        print('-' * 40)
    print()

Document similarity analysis using Hellinger-Bhattacharya distance:
Document 1: The fox is definitely smarter than the dog
Top 2 similar docs
----------------------------------------
Doc num: 8: Distance score: 0.0
Doc: The dog is smarter than the fox
----------------------------------------
Doc num: 7: Distance score: 0.96
Doc: The fox is quicker than the lazy dog
----------------------------------------

Document 2: Java is a static typed programming language unlike Python
Top 2 similar docs
----------------------------------------
Doc num: 5: Distance score: 0.53
Doc: Python and Java are popular Programming languages
----------------------------------------
Doc num: 4: Distance score: 0.766
Doc: Python is a great programming language
----------------------------------------

Document 3: I love to relax under the beautiful blue sky!
Top 2 similar docs
----------------------------------------
Doc num: 2: Distance score: 0.0
Doc: The sky is blue and beautiful
--------------------------

# Okapi BM25 ranking

BM stands for best matching. The Okapi BM25 can be formally defined as a document ranking and retrieval function based on a Bag-of-Words based model for retrieving relevant documents based on a user input query.

In [56]:
import scipy.sparse as sp

def compute_corpus_term_idfs(corpus_features, norm_corpus):
    dfs = np.diff(sp.csc_matrix(corpus_features, copy=True).indptr)
    dfs = 1 + dfs # To smoothen the idf later.
    total_docs = 1 + len(norm_corpus)
    idfs = 1.0 + np.log(float(total_docs) / dfs)
    return idfs

In [63]:
def compute_bm25_similarity(doc_features, corpus_features,
                            corpus_doc_lengths, 
                            avg_doc_length,
                            term_idfs, k1=1.5, b=0.75, top_n=3):
    # Get corpus bag of words features.
    corpus_features = corpus_features.toarray()
    
    # Convert query document features to binary features.
    # This is to keep a note of which terms exist per document.
    doc_features = doc_features.toarray()[0]
    doc_features[doc_features >= 1] = 1
    
    # Compute the document idf scores for present terms.
    doc_idfs = doc_features * term_idfs
    
    # Compute the numerator expression in BM25 equation.
    numerator_coeff = corpus_features * (k1 + 1)
    numerator = np.multiply(doc_idfs, numerator_coeff)
    
    # Compute denominator expression in BM25 equation.
    denominator_coeff = k1 * (1 - b + (b * (corpus_doc_lengths / avg_doc_length)))
    denominator_coeff = np.vstack(denominator_coeff)
    denominator = corpus_features + denominator_coeff
    
    # Compute the BM25 score combining the above equations.
    bm25_scores = np.sum(np.divide(numerator,
                                   denominator),
                         axis=1)
    
    # Get top n relevant docs with highest BM25 score.
    top_docs = bm25_scores.argsort()[::-1][:top_n]
    top_docs_with_score = [(index, round(bm25_scores[index], 3))
                           for index in top_docs]
    return top_docs_with_score

In [64]:
# Build bag of words based features first.
vectorizer, corpus_features = build_feature_matrix(norm_corpus, feature_type='frequency')
query_docs_features = vectorizer.transform(norm_query_docs)

# Get average document length of the corpus (avgdl)
docs_length = [len(doc.split()) for doc in norm_corpus]
avg_dl = np.average(docs_length)

# Get the corpus term idfs.
corpus_term_idfs = compute_corpus_term_idfs(corpus_features, norm_corpus)

# Analyze document similarity using BM25 framework.
print('Document similarity analysis using BM25')
print('=' * 60)

for index, doc in enumerate(query_docs):
    doc_features = query_docs_features[index]
    top_similar_docs = compute_bm25_similarity(doc_features, 
                                               corpus_features,
                                               docs_length,
                                               avg_dl,
                                               corpus_term_idfs,
                                               k1=1.5, b=0.75,
                                               top_n=2)
    print(f'Document {index+1}: {doc}')
    print(f'Top {len(top_similar_docs)} similar docs:')
    print('-' * 40)
    for doc_index, sim_score in top_similar_docs:
        print(f'Doc num: {doc_index+1} BM25 Score: {sim_score}\nDoc: {toy_corpus[doc_index]}')
        print('-' * 40)
    print()

Document similarity analysis using BM25
Document 1: The fox is definitely smarter than the dog
Top 2 similar docs:
----------------------------------------
Doc num: 8 BM25 Score: 7.334
Doc: The dog is smarter than the fox
----------------------------------------
Doc num: 7 BM25 Score: 3.88
Doc: The fox is quicker than the lazy dog
----------------------------------------

Document 2: Java is a static typed programming language unlike Python
Top 2 similar docs:
----------------------------------------
Doc num: 5 BM25 Score: 7.248
Doc: Python and Java are popular Programming languages
----------------------------------------
Doc num: 6 BM25 Score: 6.042
Doc: Among Programming languages, both Python and Java are the most used in Analytics
----------------------------------------

Document 3: I love to relax under the beautiful blue sky!
Top 2 similar docs:
----------------------------------------
Doc num: 2 BM25 Score: 7.334
Doc: The sky is blue and beautiful
-----------------------------